In [ ]:
import pandas as pd

In [ ]:
df_train = pd.read_csv('train_5CLrC8b/train.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
df_train['source_'] = 1
df_test['source_'] =0
df = pd.concat([df_train,df_test])
df.reset_index(inplace=True)
df.drop(['index'],inplace=True,axis=1)
df.head()

In [ ]:
#x = pd.to_datetime(df['checkout_date']) - pd.to_datetime(df['checkin_date'])

In [ ]:
#df['num_days'] = x.dt.days

In [ ]:
df.drop(['checkout_date','checkin_date','booking_date','memberid','reservation_id',
        'reservationstatusid_code' ,'season_holidayed_code','state_code_residence'],inplace=True,axis=1)

In [ ]:
df.head()

In [ ]:
# l = ['booking_type_code',
#        'channel_code', 'cluster_code', 'main_product_code',
#        'member_age_buckets', 'numberofadults', 'numberofchildren','resort_id',
#        'resort_region_code', 'resort_type_code', 'room_type_booked_code',
#        'roomnights', 'persontravellingid',
#         'state_code_resort']
# df = pd.get_dummies(df,columns=l)

In [ ]:
df.head()

In [ ]:
preprocessed_main_test = df.loc[df['source_'] == 0]
preprocessed_main_train = df.loc[df['source_'] == 1]

In [ ]:
preprocessed_main_test.drop(['source_','amount_spent_per_room_night_scaled'],axis =1,inplace=True)

In [ ]:
y_train = (preprocessed_main_train['amount_spent_per_room_night_scaled'])
x_train = preprocessed_main_train.drop(['amount_spent_per_room_night_scaled','source_'],axis =1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train_internal, y_test_internal = train_test_split(x_train,y_train ,test_size =.2)


In [ ]:
       
params = {
'num_leaves': 60,
'max_bin': 110,
'min_data_in_leaf': 50,
'learning_rate': 0.01,
'min_sum_hessian_in_leaf': 0.000446,
'bagging_fraction': 0.60,
'bagging_freq': 15,
'max_depth': 20,
'save_binary': True,
'seed': 31452,
'feature_fraction_seed': 31415,
'feature_fraction': 0.51,
'bagging_seed': 31415,
'drop_seed': 31415,
'data_random_seed': 31415,
'objective': 'regression',
'boosting_type': 'gbdt',
'verbose': 1,
'metric': 'rmse',
'is_unbalance': False,
#     'categorical_feature': "name:branch_id"
# 'scale_pos_weight': len(train_df["target"] == 0) / sum(train_df["target"])
}

# Import the model we are using
#from sklearn.ensemble import RandomForestClassifier
# from tpot import TPOTClassifier
import lightgbm as lgb

lg =  lgb.LGBMRegressor()
lg.fit(X_train, y_train_internal)
d_train = lgb.Dataset(X_train, label=y_train_internal)
# Instantiate model with 1000 decision trees
#rf = RandomForestClassifier()

# Train the model on training data
#rm = lgb.tr(X_train, y_train_internal)



#clf = lgb.train(params, d_trainnames=['train','valid'], evals_result=evals_results, num_boost_round=3000,
             

#n_estimators = clf.best_iteration
print('stert')


#clf = lgb.train(params, d_train,30000)

In [ ]:
pred = lg.predict(preprocessed_main_test)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
pred_orig_test = lg.predict(X_test)
rms = sqrt(mean_squared_error(y_test_internal, pred_orig_test))
rms

In [ ]:
#pred = regr.predict(preprocessed_main_test)
final_res = list(zip(df_test['reservation_id'],pred))

In [ ]:
res_ = pd.DataFrame(final_res,columns=['reservation_id','amount_spent_per_room_night_scaled'])

In [ ]:
res_.to_csv('sub_2_lgb.csv',index=False)